# Procesando tweets con Kafka

El objetivo del ejercicio es procesar los datos de la API de streaming de Tweeter para obtener el tending topic de la última hora. 

## Procesar datos de la API de streaming de Tweeter

Vamos a emular la api de streaming de Tweeter. Para ellos nos vamos a valer de un ficheo en el que se han escrito todos los mensajes que nos ha devuelto la api de tweeter durante una hora.

Iremos insertando esos mensajes en Kafka para procesarlos posteriormente en real time.

In [ ]:
#!pip install kafka-python
import json
from kafka import KafkaProducer

def insert_tweet(tweet_json):
    json_producer = KafkaProducer(bootstrap_servers='localhost:9092',
                                  value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    json_producer.send('tweetapi', tweet_json)
    json_producer.flush()
    

In [ ]:
#from pprintpp import pprint as pp
import sys

tweets_data_path = '../data/tweets.json'

tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    tweet_json = json.loads(line)
    insert_tweet(tweet_json)

## Leemos los mensajes de la cola de Kafka según llegan y los prcesamos

In [ ]:
import pandas as pd
from kafka import KafkaConsumer

hashtags = []

consumer = KafkaConsumer('jsontest', 
                         bootstrap_servers = ['localhost:9092'],
                         value_deserializer = lambda m: json.loads(m.decode('utf-8')),
                         consumer_timeout_ms = 10000,
                         auto_offset_reset = 'earliest', 
                         enable_auto_commit = True)
for msg in consumer:
    entities = msg.value['entities']
    for entity in entities:
        hashtags.append((entity.lower(), 1))

columns = ['hashtag', 'count']
df = pd.DataFrame(hashtags, columns = columns)
df2 = df.groupby(['hashtag']).sum()
sorted_values = df2.sort_values(by = ['count'], ascending = False)
print(sorted_values.head(10))